In [1]:
import pandas as pd
import numpy as np
from torchsummary import summary

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
csvfile = '/content/drive/MyDrive/Projet-JDS/SMOTEd.csv'
smoted = pd.read_csv(csvfile)

In [4]:
csv1 = '/content/drive/MyDrive/Projet-JDS/non_accident.csv'
csv = pd.read_csv(csv1)

# Transformation aux numéros

In [5]:
csv['WEATHER_CONDITION']= csv['WEATHER_CONDITION'].replace({'CLEAR':1,'RAIN':2,'CLOUDY/OVERCAST':3,'SNOW':4})
csv['LIGHTING_CONDITION']= csv['LIGHTING_CONDITION'].replace({'DAYLIGHT':1,'DARKNESS, LIGHTED ROAD':2,'DARKNESS':3,'DUSK':4,'DAWN':5})
csv['LIGHTING_CONDITION'].value_counts()

1    370507
2    212451
3     81013
4     62406
5     57775
Name: LIGHTING_CONDITION, dtype: int64

In [6]:
csv['ROADWAY_SURFACE_COND']= csv['ROADWAY_SURFACE_COND'].replace({'DRY':1,'WET':2,'SNOW OR SLUSH':3})
csv['ROADWAY_SURFACE_COND'].value_counts()

1    509496
2    140506
3    134150
Name: ROADWAY_SURFACE_COND, dtype: int64

In [9]:
y = csv['accident'] #separating the target variable
X = csv.drop(['accident',csv.columns[0]],axis=1)

In [10]:
#%%time 
from imblearn.over_sampling import SMOTENC

# By default, the values sampled are continous. We have to explicitely mention if we deal with categorical data
sample = SMOTENC(random_state=2,categorical_features=[0,1,2,4,5,6,7],n_jobs=-1,sampling_strategy='minority')
X_2, y_2 = sample.fit_resample(X, y.ravel())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#from imblearn.under_sampling import ClusterCentroids
#cc = ClusterCentroids(random_state=0)
#X_resampled, y_resampled = cc.fit_resample(X, y)

In [ ]:
#y.value_counts()[0]/np.sum(y.value_counts())

In [11]:
print('The size of the generated dataset from Louis: {}' .format(len(y)))
print('In that how much is accident: {}'.format(sum(y)))
print('In that how much is non-accident: {}'.format(len(y) - sum(y)))
print('The percentage of non-accident data: {}%'.format(y.value_counts()[0]/np.sum(y.value_counts())*100))

The size of the generated dataset from Louis: 784152
In that how much is accident: 418560
In that how much is non-accident: 365592
The percentage of non-accident data: 46.62259357879594%


In [12]:
train_orig = pd.read_csv('/content/drive/MyDrive/Projet-JDS/train_orig.csv') #Original dataset

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,7,8,10,11,12,13,14,15,17,18,24,25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
train_orig['WEATHER_CONDITION'].value_counts()

CLEAR              515099
RAIN                59718
SNOW                20040
CLOUDY/OVERCAST     20027
Name: WEATHER_CONDITION, dtype: int64

# Neural Networks

In [14]:
import torch
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torch.utils.data.sampler as sampler
from torchsummary import summary
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device '+str(device))

Using device cuda:0


In [15]:
smoted = pd.read_csv('/content/drive/MyDrive/Projet-JDS/SMOTEd.csv')

In [16]:
y_2 = smoted['accident']
X_2 = smoted.drop(['accident'],axis=1)

In [17]:
X_2 #sanity check

,WEATHER_CONDITION,LIGHTING_CONDITION,CRASH_WEEKDAY,CRASH_HOUR,CRASH_Month,x,y,ROADWAY_SURFACE_COND
0,1,1,3,7,7,43.0,43.0,1
1,1,1,4,16,5,38.0,16.0,1
2,1,1,1,8,5,36.0,35.0,1
3,1,2,6,1,6,14.0,49.0,1
4,3,1,1,16,8,8.0,47.0,1
...,...,...,...,...,...,...,...,...
837115,1,2,1,21,1,37.0,65.0,1
837116,4,1,4,19,8,40.0,13.0,3
837117,1,2,4,5,3,38.0,47.0,3
837118,3,3,2,14,5,30.0,61.0,1


# Preparing to split the dataset

In [18]:
idx= np.random.randint(0,len(smoted),size=len(smoted))

In [19]:
# 90% 5% 5%
train_size = 0.9
valid_size = 0.05
test_size = 0.05

In [20]:
X_test,y_test = X_2.iloc[idx[0:int(test_size*len(idx))]], y_2.iloc[idx[0:int(test_size*len(idx))]]
X_train,y_train = X_2.iloc[idx[int((test_size*len(idx))) : len(X_test) + int(train_size*len(idx))]], y_2.iloc[idx[int((test_size*len(idx))) : len(X_test) + int(train_size*len(idx))]]
X_valid,y_valid = X_2.iloc[idx[ len(X_test) + int(train_size*len(idx)) :]], y_2.iloc[idx[ len(X_test) + int(train_size*len(idx)) :]]

In [36]:
y_valid

656331    0
549984    0
825164    0
111152    1
34834     1
         ..
172789    1
64193     1
765178    0
669738    0
529410    0
Name: accident, Length: 41856, dtype: int64

In [21]:

tensor_xtrain = torch.Tensor(X_train.values) # transformation to torch tensor
tensor_ytrain = torch.Tensor(y_train.values)

traingrouping = TensorDataset(tensor_xtrain,tensor_ytrain) # creating datset
train_loader = DataLoader(traingrouping,batch_size=128) # creating dataloader

In [22]:
tensor_xvalid = torch.Tensor(X_valid.values) # transformation to torch tensor
tensor_yvalid = torch.Tensor(y_valid.values)

validgrouping = TensorDataset(tensor_xvalid,tensor_yvalid) # creating datset
valid_loader = DataLoader(validgrouping,batch_size=128) # creating dataloader

In [23]:
tensor_xtest = torch.Tensor(X_test.values) # transform to torch tensor
tensor_ytest = torch.Tensor(y_test.values)

testgrouping = TensorDataset(tensor_xtest,tensor_ytest) # creating datset
test_loader = DataLoader(testgrouping,batch_size=128) # creating dataloader

In [24]:
class neuralnetwork(nn.Module):
  def __init__(self):
        super(neuralnetwork, self).__init__()
        self.fc1 = nn.Linear(8,100)
        self.fc2 = nn.Linear(100,256)
        self.fc3 = nn.Linear(256,256)
        self.fc4 = nn.Linear(256,2)
      #  self.dropout = nn.Dropout(p=0.15)
  def forward(self,x):
        x = F.relu(self.fc1(x))
        x = (F.relu(self.fc2(x)))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [25]:
model = neuralnetwork()
model.to(device=device)
summary(model,(1,8))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 100]             900
            Linear-2               [-1, 1, 256]          25,856
            Linear-3               [-1, 1, 256]          65,792
            Linear-4                 [-1, 1, 2]             514
Total params: 93,062
Trainable params: 93,062
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.36
Estimated Total Size (MB): 0.36
----------------------------------------------------------------


#optimizer

In [26]:
#optimizer = optim.Adam(model.parameters(),lr=0.01)
optimizer = optim.SGD(model.parameters(), lr=0.001,momentum=0.9)

# Criterion: Loss function

In [27]:
lossfunction = nn.CrossEntropyLoss()

In [28]:
def training(n_epochs, train_loader, valid_loader, model, criterion, optimizer):

  train_losses, valid_losses = [], []

  valid_loss_min = np.Inf

  for epoch in range(n_epochs):
      train_loss, valid_loss = 0, 0

      model.train()
      for data, label in train_loader:
          data = data.to(device=device, dtype=torch.float32)
          label = label.to(device=device, dtype=torch.long)
          optimizer.zero_grad()
          output = model(data)
          loss = criterion(output, label)
          loss.backward()
          optimizer.step()
          train_loss += loss.item() * data.size(0)
      
      model.eval()
      for data, label in valid_loader:
          data = data.to(device=device, dtype=torch.float32)
          label = label.to(device=device, dtype=torch.long)
          with torch.no_grad():
              output = model(data)
          loss = criterion(output,label)
          valid_loss += loss.item() * data.size(0)
  
      train_loss /= len(train_loader.sampler)
      valid_loss /= len(valid_loader.sampler)
      train_losses.append(train_loss)
      valid_losses.append(valid_loss)
      
      print('epoch: {} \ttraining Loss: {:.6f} \tvalidation Loss: {:.6f}'.format(epoch+1, train_loss, valid_loss))

      # save model if validation loss has decreased
      if valid_loss <= valid_loss_min:
          print('validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
          valid_loss_min,
          valid_loss))
          torch.save(model.state_dict(), 'model.pt')
          valid_loss_min = valid_loss
      
  return train_losses, valid_losses

In [29]:

train_losses, valid_losses = training(50, train_loader, valid_loader, model, lossfunction, optimizer)

epoch: 1 	training Loss: 0.607420 	validation Loss: 0.577233
validation loss decreased (inf --> 0.577233).  Saving model ...
epoch: 2 	training Loss: 0.526167 	validation Loss: 0.485656
validation loss decreased (0.577233 --> 0.485656).  Saving model ...
epoch: 3 	training Loss: 0.473376 	validation Loss: 0.459419
validation loss decreased (0.485656 --> 0.459419).  Saving model ...
epoch: 4 	training Loss: 0.446854 	validation Loss: 0.447489
validation loss decreased (0.459419 --> 0.447489).  Saving model ...
epoch: 5 	training Loss: 0.430767 	validation Loss: 0.446710
validation loss decreased (0.447489 --> 0.446710).  Saving model ...
epoch: 6 	training Loss: 0.418884 	validation Loss: 0.429499
validation loss decreased (0.446710 --> 0.429499).  Saving model ...
epoch: 7 	training Loss: 0.408381 	validation Loss: 0.411018
validation loss decreased (0.429499 --> 0.411018).  Saving model ...
epoch: 8 	training Loss: 0.399193 	validation Loss: 0.411198
epoch: 9 	training Loss: 0.390666 

In [30]:


def evaluation(model, test_loader, criterion, detail): 

  test_loss = 0.0
  class_correct = list(0. for i in range(2))
  class_total = list(0. for i in range(2))
  class_names = ['class {}'.format(i) for i in range(2)]
  print(class_names)
  model.eval()
  for data, label in test_loader:

      data = data.to(device=device, dtype=torch.float32)
      label = label.to(device=device, dtype=torch.long)

      with torch.no_grad():
          output = model(data)
      loss = criterion(output, label)
      test_loss += loss.item()*data.size(0)
      _, pred = torch.max(output, 1)
      correct = np.squeeze(pred.eq(label.data.view_as(pred)))

      for i in range(len(label)):
          digit = label.data[i]
          class_correct[digit] += correct[i].item()
          class_total[digit] += 1

  test_loss = test_loss/len(test_loader.sampler)
  print('test Loss: {:.6f}\n'.format(test_loss))
  if detail :
    for i in range(2):
      if class_total[i] != 0:
        print('test accuracy of %s: %2d%% (%2d/%2d)' % (class_names[i], 100 * class_correct[i] / class_total[i], np.sum(class_correct[i]), np.sum(class_total[i])))
      #else:
        #print('test accuracy of {}: 0/0 (no sample in this class)'.format(i))

  print('\n test accuracy (overall): %2.2f%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total), np.sum(class_correct), np.sum(class_total)))

In [31]:
evaluation(model,test_loader,lossfunction,True)

['class 0', 'class 1']
test Loss: 0.334596

test accuracy of class 0: 81% (17063/20892)
test accuracy of class 1: 88% (18642/20964)

 test accuracy (overall): 85.30% (35705/41856)


In [32]:
evaluation(model,valid_loader,lossfunction,True)

['class 0', 'class 1']
test Loss: 0.341421

test accuracy of class 0: 81% (17082/20980)
test accuracy of class 1: 88% (18445/20876)

 test accuracy (overall): 84.88% (35527/41856)


# Random Forest

In [38]:
from sklearn.ensemble import RandomForestClassifier

In [47]:

rf = RandomForestClassifier()
modelRF = rf.fit(X_train.values,y_train.values)

In [49]:
testpredicted = rf.predict(X_test)

print( ( len(testpredicted[y_test==0]) - np.sum(testpredicted[y_test==0]) ) / len(testpredicted[y_test==0]) )
print( np.sum(testpredicted[y_test==1]) / len(testpredicted[y_test==1]) )

0.933850277618227
0.9559244419004007


In [53]:
validpredicted = modelRF.predict(X_valid)

print( ( len(testpredicted[y_test==0]) - np.sum(testpredicted[y_test==0]) ) / len(testpredicted[y_test==0]) )
print( np.sum(testpredicted[y_test==1]) / len(testpredicted[y_test==1]) )

0.933850277618227
0.9559244419004007


In [54]:
np.sum(testpredicted==y_test)/len(y_test)

0.9449063455657493

In [55]:
np.sum(validpredicted==y_valid)/len(y_valid)


0.9449780198776758

In [56]:
validpredicted = modelRF.predict(X_valid.values)
print( ( len(validpredicted[y_valid==0]) - np.sum(validpredicted[y_valid==0]) ) / len(validpredicted[y_valid==0]) )
print( np.sum(validpredicted[y_valid==1]) / len(validpredicted[y_valid==1]) )

0.9367016205910391
0.9532956505077601


In [57]:
modelRF.predict_proba([[ 2.,  1.,  5., 15.,  5., 20., 7.,  2.]])

array([[0.53, 0.47]])

In [58]:
X_test.values[-2]

array([ 1.,  5.,  3., 17.,  8., 20., 56.,  1.])

# SVM

In [59]:
from sklearn.svm import LinearSVC

In [60]:
modelsvm = LinearSVC(max_iter =5000)
modelsvm.fit(X_train.values, y_train.values)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=5000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [61]:
predictedsvmtest = modelsvm.predict(X_test.values)

In [62]:
np.sum( (predictedsvmtest==y_test).values )/len(y_test)

0.6293482415902141

#Validation set testing - Linear SVM

In [63]:
predictedsvmvalid = modelsvm.predict(X_valid.values)
np.sum( (predictedsvmvalid==y_valid).values )/len(y_valid)

0.627723623853211

In [64]:
predictedsvmtrain = modelsvm.predict(X_train.values)
np.sum( (predictedsvmtrain==y_train).values )/len(y_train)

0.6282001252973156